<a href="https://colab.research.google.com/github/AshOne91/PLAYDATA-COLAB/blob/main/%EB%AA%A8%EB%8D%B8_%EC%84%B1%EB%8A%A5_%EA%B3%A0%EB%8F%84%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See

In [ ]:
# 데이터는 mnist
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor, Compose
# 데이터 전처리
transforms = Compose([ToTensor()])
train_data = MNIST(root='./', download=True, transform=transforms, train=True)
test_data =  MNIST(root='./', download=True, transform=transforms,train=False)
train_data.data.shape

torch.Size([60000, 28, 28])

In [ ]:
# 데이터는 2D형태로 변환
import torch
X_train = torch.flatten(train_data.data, start_dim=1)  # torch.flatten(train_data.data[0])
y_train = train_data.targets
X_test = torch.flatten(test_data.data, start_dim=1)
y_test = test_data.targets

X_train.shape, y_train.shape,  X_test.shape, y_test.shape

(torch.Size([60000, 784]),
 torch.Size([60000]),
 torch.Size([10000, 784]),
 torch.Size([10000]))

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd

In [ ]:
# dataframe 으로 변환
train_df = pd.DataFrame(X_train)
train_df['target'] = y_train
test_df = pd.DataFrame(X_test)
test_df['target'] = y_test

In [ ]:
# TabularDataset
train_data = TabularDataset(train_df)
test_data = TabularDataset(test_df)

In [ ]:
# 시간설정 (10분)
time_limit = 60*10 # 1h : 60*60
# gpu 사용불가 모델은 제외
eval_metric = 'f1_weighted'  # 평가방법  이진분류는 roc_auc  다중분류 roc_auc_ovo
excluded_model_types = [
    'KNN','RF','XT','LR','NN'
]
# AutoGluon 학습
predictor = TabularPredictor(label='target',path='AutogluonModels/',eval_metric=eval_metric) \
.fit(train_data,time_limit = time_limit,excluded_model_types=excluded_model_types)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.85 GB / 12.67 GB (85.6%)
Disk Space Avail:   65.50 GB / 112.64 GB (58.1%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='high'         : Strong accuracy with fast inference spe

In [ ]:
leaderboard = predictor.leaderboard(silent=True)
leaderboard

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.988003,f1_weighted,1.306103,601.223051,0.002548,0.121812,2,True,3
1,NeuralNetFastAI,0.987198,f1_weighted,0.110458,106.030788,0.110458,106.030788,1,True,1
2,LightGBMXT,0.984010,f1_weighted,1.193097,495.070451,1.193097,495.070451,1,True,2


In [ ]:
predictor.feature_importance(train_data)

These features in provided data are not utilized by the predictor and will be ignored: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 20

KeyError: "644 required columns are missing from the provided dataset to transform using AutoMLPipelineFeatureGenerator. 644 missing columns: ['12', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '113', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '171', '172', '173', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '198', '199'

In [ ]:
# bestmodel
best_model_name = predictor.model_best  #
print(f'best model name : {best_model_name}')
# 테스트데이터
predcitions = predictor.predict(test_data)
predcitions, y_test

best model name : WeightedEnsemble_L2


/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


(0       7
 1       2
 2       1
 3       0
 4       4
        ..
 9995    2
 9996    3
 9997    4
 9998    5
 9999    6
 Name: target, Length: 10000, dtype: int64,
 tensor([7, 2, 1,  ..., 4, 5, 6]))

In [ ]:
# 성능이 2번째인 모델로 예측
leaderboard.iloc[1]['model']
predictions_2 = predictor.predict(test_data,model=leaderboard.iloc[1]['model'])
predictions_2, y_test

/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


(0       7
 1       2
 2       1
 3       0
 4       4
        ..
 9995    2
 9996    3
 9997    4
 9998    5
 9999    6
 Name: target, Length: 10000, dtype: int64,
 tensor([7, 2, 1,  ..., 4, 5, 6]))

In [ ]:
from sklearn.metrics import f1_score
metrics = predictor.evaluate(test_data)
metrics

/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


{'f1_weighted': 0.9830995276070795,
 'accuracy': 0.9831,
 'balanced_accuracy': 0.982962084803263,
 'mcc': 0.9812157308479809}

In [ ]:
# 캘리포니아. 보스턴 집값 예측
# 데이터셋 만들기 - automl
# 분류모델인지 회귀인지는 디폴트로 auto 판단
# 시간은 20분 설정  모델 리더보드를 통해 확인

In [ ]:
# 보스턴 집값 데이터 로드
# from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
boston_X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
boston_y = raw_df.values[1::2, 2]

from sklearn.datasets import fetch_california_housing
california =  fetch_california_housing()
californa_X = california.data
californa_y = california.target

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
from sklearn.model_selection import train_test_split

def makeDf(X,y):
  df = pd.DataFrame(X)
  df['target'] = y
  return df

boston_df = makeDf(boston_X,boston_y)
californa_df = makeDf(californa_X,californa_y)

boston_df_train,boston_df_test  =  train_test_split(boston_df,random_state=42)
californa_df_train,californa_df_test =  train_test_split(californa_df,random_state=42)

# TabularDataset
boston_train = TabularDataset(boston_df_train)
boston_test = TabularDataset(boston_df_test)

californa_train = TabularDataset(californa_df_train)
californa_test = TabularDataset(californa_df_test)

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
df = scaler.fit_transform(boston_df.drop(columns=['target']))
df = pd.DataFrame(df)
df['target'] = boston_df['target']
boston_train = df
eval_metric = 'rmse'
problem_type="regression"
excluded_model_types = [
    'KNN','RF','XT','LR','NN'
]
time_limit = 60*10
boston_predictor = TabularPredictor(
    label='target',path='AutogluonModels/',eval_metric=eval_metric,problem_type=problem_type) \
.fit(boston_train,time_limit = time_limit
     ,excluded_model_types=excluded_model_types
     ,presets='best_quality'
     )

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       9.72 GB / 12.67 GB (76.7%)
Disk Space Avail:   61.19 GB / 107.72 GB (56.8%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked 

In [ ]:
scaler = RobustScaler()
df = scaler.fit_transform(boston_test.drop(columns=['target']))
df = pd.DataFrame(df)
df['target'] = boston_test['target']
boston_test = df
boston_test.dropna(inplace=True)
boston_predictor.evaluate(boston_test)

/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")
/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you t

{'root_mean_squared_error': -10.016515335549649,
 'mean_squared_error': -100.3305794673013,
 'mean_absolute_error': -7.324061626858182,
 'r2': -3.1340773148114964,
 'pearsonr': 0.04058070473019103,
 'median_absolute_error': -5.826446151733398}

In [ ]:
# 학습시간
# 시간설정 (20분)
time_limit = 60*20 # 1h : 60*60
# gpu 사용불가 모델은 제외
eval_metric = 'rmse'  # 평가방법  이진분류는 roc_auc  다중분류 roc_auc_ovo
problem_type="regression"
excluded_model_types = [
    'KNN','RF','XT','LR','NN'
]
# AutoGluon 학습
boston_predictor = TabularPredictor(
    label='target',path='AutogluonModels/',eval_metric=eval_metric,problem_type=problem_type) \
.fit(boston_train,time_limit = time_limit
     ,excluded_model_types=excluded_model_types
     ,presets='best_quality'
     )
californa_predictor = TabularPredictor(
    label='target',path='AutogluonModels/',eval_metric=eval_metric,problem_type=problem_type) \
.fit(californa_train,time_limit = time_limit
     ,excluded_model_types=excluded_model_types
     ,presets='best_quality'
     )
# 평가방법

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       8.08 GB / 12.67 GB (63.7%)
Disk Space Avail:   61.40 GB / 107.72 GB (57.0%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked 

(autoscaler +3m18s) Warning: The following resource request cannot be scheduled right now: {'CPU': 2.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(_ray_fit pid=13864) [1000]	valid_set's rmse: 2.75178
(_ray_fit pid=13864) [2000]	valid_set's rmse: 2.70572
(_ray_fit pid=13998) [1000]	valid_set's rmse: 4.89045
(_ray_fit pid=14094) [1000]	valid_set's rmse: 3.12821


(_dystack pid=13698) 	-3.4301	 = Validation score   (-root_mean_squared_error)
(_dystack pid=13698) 	38.85s	 = Training   runtime
(_dystack pid=13698) 	0.05s	 = Validation runtime
(_dystack pid=13698) Fitting model: LightGBM_BAG_L1 ... Training model for up to 53.24s of the 103.23s of remaining time.
(_dystack pid=13698) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=0.03%)
(_dystack pid=14201) Running DyStack sub-fit ...
(_dystack pid=14201) Beginning AutoGluon training ... Time limit = 300s
(_dystack pid=14201) AutoGluon will save models to "/content/AutogluonModels/ds_sub_fit/sub_fit_ho"
(_dystack pid=14201) Train Data Rows:    336
(_dystack pid=14201) Train Data Columns: 13
(_dystack pid=14201) Label Column:       target
(_dystack pid=14201) Problem Type:       regression
(_dystack pid=14201) Preprocessing data ...
(_dystack pid=14201) Using Feature Generators to preprocess the data ...
(_dystack pid=142

(autoscaler +3m43s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


(_dystack pid=14201) Fitting model: LightGBMXT_BAG_L1 ... Training model for up to 199.92s of the 299.94s of remaining time.
(_dystack pid=14201) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=0.03%)


...

(_ray_fit pid=15597) 	Ran out of time, early stopping on iteration 2144.


(autoscaler +6m13s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


(_dystack pid=14201) 	-3.2826	 = Validation score   (-root_mean_squared_error)
(_dystack pid=14201) 	59.79s	 = Training   runtime
(_dystack pid=14201) 	0.02s	 = Validation runtime
(_dystack pid=14201) Fitting model: NeuralNetFastAI_BAG_L1 ... Training model for up to 20.26s of the 120.28s of remaining time.
(_ray_fit pid=15906) 	Ran out of time, early stopping on iteration 3552.
(_dystack pid=14201) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=0.00%)


(autoscaler +6m48s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


(_dystack pid=14201) 	-3.7689	 = Validation score   (-root_mean_squared_error)
(_dystack pid=14201) 	48.99s	 = Training   runtime
(_dystack pid=14201) 	0.13s	 = Validation runtime
	[Errno 2] No such file or directory: '/content/AutogluonModels/ds_sub_fit/sub_fit_ho/models/LightGBMXT_BAG_L1/model.pkl'
	1	 = Optimal   num_stack_levels (Stacked Overfitting Occurred: False)
	259s	 = DyStack   runtime |	941s	 = Remaining runtime
Starting main fit with num_stack_levels=1.
	For future fit calls on this dataset, you can skip DyStack to save time: `predictor.fit(..., dynamic_stacking=False, num_stack_levels=1)`
Beginning AutoGluon training ... Time limit = 941s
AutoGluon will save models to "/content/AutogluonModels"
Train Data Rows:    379
Train Data Columns: 13
Label Column:       target
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    9297.82 MB
	Train Data

In [ ]:
boston_predictor.leaderboard(silent=True)

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-2.733966,root_mean_squared_error,0.184994,272.016279,0.000674,0.016376,2,True,11
1,WeightedEnsemble_L3,-2.746982,root_mean_squared_error,0.676790,465.464322,0.000580,0.020603,3,True,18
2,NeuralNetTorch_BAG_L1,-2.895835,root_mean_squared_error,0.159194,126.800891,0.159194,126.800891,1,True,6
3,NeuralNetTorch_r79_BAG_L1,-2.931230,root_mean_squared_error,0.231888,81.423590,0.231888,81.423590,1,True,9
4,NeuralNetTorch_BAG_L2,-2.965657,root_mean_squared_error,0.748815,437.014513,0.208993,64.434182,2,True,17
5,CatBoost_BAG_L2,-2.992598,root_mean_squared_error,0.593412,430.845202,0.053590,58.264871,2,True,14
6,LightGBM_BAG_L2,-3.026082,root_mean_squared_error,0.622619,407.178847,0.082798,34.598516,2,True,13
7,LightGBMXT_BAG_L2,-3.042896,root_mean_squared_error,0.692327,411.419656,0.152506,38.839325,2,True,12
8,NeuralNetFastAI_BAG_L2,-3.189674,root_mean_squared_error,0.673627,420.968640,0.133806,48.388309,2,True,15
9,CatBoost_BAG_L1,-3.285920,root_mean_squared_error,0.025126,145.199012,0.025126,145.199012,1,True,3


In [ ]:
predict = boston_predictor.predict(boston_test.drop(columns=['target']))
mean_target = boston_test['target'].mean()
mean_std = boston_test['target'].std()
predictions_rescaled = predict*mean_std + mean_target
from sklearn.metrics import r2_score
r2_score(boston_df_test['target'],predictions_rescaled)

-4.3792101823255

In [ ]:
californa_predictor.leaderboard(silent=True)

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-0.433024,root_mean_squared_error,20.842119,800.723066,0.000685,0.028901,3,True,10
1,NeuralNetFastAI_BAG_L2,-0.434436,root_mean_squared_error,20.680544,727.866728,0.615121,146.401626,2,True,9
2,WeightedEnsemble_L2,-0.435252,root_mean_squared_error,5.050797,399.785899,0.000998,0.027942,2,True,5
3,CatBoost_BAG_L1,-0.437094,root_mean_squared_error,0.658509,344.306607,0.658509,344.306607,1,True,3
4,CatBoost_BAG_L2,-0.437902,root_mean_squared_error,20.125776,618.816718,0.060353,37.351616,2,True,8
5,LightGBM_BAG_L2,-0.438613,root_mean_squared_error,20.165960,616.940922,0.100537,35.475820,2,True,7
6,LightGBMXT_BAG_L2,-0.442495,root_mean_squared_error,20.373497,618.568178,0.308074,37.103076,2,True,6
7,LightGBM_BAG_L1,-0.448945,root_mean_squared_error,4.391290,55.451350,4.391290,55.451350,1,True,2
8,LightGBMXT_BAG_L1,-0.468873,root_mean_squared_error,14.443148,95.814415,14.443148,95.814415,1,True,1
9,NeuralNetFastAI_BAG_L1,-0.634110,root_mean_squared_error,0.572476,85.892730,0.572476,85.892730,1,True,4


In [ ]:
californa_predictor.predict(californa_test), californa_test['target']

/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")
/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you t

(20046    0.558168
 3024     0.848678
 15663    5.063264
 20484    2.537698
 9814     2.589649
            ...   
 5363     4.865023
 19755    0.670456
 4885     1.286046
 13043    2.774793
 8583     4.231379
 Name: target, Length: 5160, dtype: float32,
 20046    0.47700
 3024     0.45800
 15663    5.00001
 20484    2.18600
 9814     2.78000
           ...   
 5363     5.00001
 19755    0.63200
 4885     1.17700
 13043    2.63100
 8583     4.81500
 Name: target, Length: 5160, dtype: float64)

In [ ]:
californa_predictor.evaluate(californa_test)

/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")
/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you t

{'root_mean_squared_error': -0.42089142491560777,
 'mean_squared_error': -0.17714959156749072,
 'mean_absolute_error': -0.27173552918817645,
 'r2': 0.8661221525690034,
 'pearsonr': 0.9307890934939936,
 'median_absolute_error': -0.17267675447463982}

Boston DataSet

In [ ]:
import pandas as pd
import numpy as np

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]
df = pd.DataFrame(X)
df['target'] = y

train_data, test_df = train_test_split(df,random_state=42)

# 회귀는 기본적으로 -RMSE를 사용
# 여러모델과 분류 및 기타 다른 용도로도 사용하기 때문에 높을수록 성능이 우수하도록 설계
# 손실값은 - 로 표현
predictor= TabularPredictor(label ='target').fit(train_data = train_data, verbosity = 2,presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20250416_045631"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       9.92 GB / 12.67 GB (78.3%)
Disk Space Avail:   61.28 GB / 107.72 GB (56.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fi

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsDist_BAG_L1,-5.614085e-07,-6.577513,root_mean_squared_error,0.021015,0.013638,0.005910,0.021015,0.013638,0.005910,1,True,2
1,XGBoost_r33_BAG_L1,-5.813374e-01,-3.849863,root_mean_squared_error,1.489797,0.261932,44.530389,1.489797,0.261932,44.530389,1,True,19
2,CatBoost_r9_BAG_L1,-6.673254e-01,-3.565220,root_mean_squared_error,0.496101,0.044414,233.428024,0.496101,0.044414,233.428024,1,True,16
3,CatBoost_r13_BAG_L1,-7.363171e-01,-3.385719,root_mean_squared_error,0.432711,0.024206,172.259996,0.432711,0.024206,172.259996,1,True,23
4,XGBoost_BAG_L1,-7.762401e-01,-3.706454,root_mean_squared_error,0.338563,0.096619,23.378653,0.338563,0.096619,23.378653,1,True,9
5,LightGBMLarge_BAG_L1,-8.534829e-01,-3.711882,root_mean_squared_error,0.592724,0.060869,40.470383,0.592724,0.060869,40.470383,1,True,11
6,CatBoost_BAG_L1,-8.743485e-01,-3.285920,root_mean_squared_error,0.595561,0.029414,266.716108,0.595561,0.029414,266.716108,1,True,6
7,CatBoost_r177_BAG_L1,-9.242222e-01,-3.313882,root_mean_squared_error,0.119963,0.014112,58.107791,0.119963,0.014112,58.107791,1,True,12
8,CatBoost_r137_BAG_L1,-1.069514e+00,-3.237796,root_mean_squared_error,0.124587,0.015640,121.723618,0.124587,0.015640,121.723618,1,True,21
9,ExtraTrees_r42_BAG_L1,-1.220821e+00,-3.316153,root_mean_squared_error,0.149321,0.195372,0.963074,0.149321,0.195372,0.963074,1,True,20


In [ ]:
predictor.leaderboard(train_data, silent=True)

/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")
/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you t

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsDist_BAG_L1,-5.614085e-07,-6.577513,root_mean_squared_error,0.014902,0.013638,0.005910,0.014902,0.013638,0.005910,1,True,2
1,XGBoost_r33_BAG_L1,-5.813374e-01,-3.849863,root_mean_squared_error,0.995560,0.261932,44.530389,0.995560,0.261932,44.530389,1,True,19
2,CatBoost_r9_BAG_L1,-6.673254e-01,-3.565220,root_mean_squared_error,0.687863,0.044414,233.428024,0.687863,0.044414,233.428024,1,True,16
3,CatBoost_r13_BAG_L1,-7.363171e-01,-3.385719,root_mean_squared_error,0.114889,0.024206,172.259996,0.114889,0.024206,172.259996,1,True,23
4,XGBoost_BAG_L1,-7.762401e-01,-3.706454,root_mean_squared_error,0.541543,0.096619,23.378653,0.541543,0.096619,23.378653,1,True,9
5,LightGBMLarge_BAG_L1,-8.534829e-01,-3.711882,root_mean_squared_error,1.308167,0.060869,40.470383,1.308167,0.060869,40.470383,1,True,11
6,CatBoost_BAG_L1,-8.743485e-01,-3.285920,root_mean_squared_error,0.350428,0.029414,266.716108,0.350428,0.029414,266.716108,1,True,6
7,CatBoost_r177_BAG_L1,-9.242222e-01,-3.313882,root_mean_squared_error,0.217206,0.014112,58.107791,0.217206,0.014112,58.107791,1,True,12
8,CatBoost_r137_BAG_L1,-1.069514e+00,-3.237796,root_mean_squared_error,0.096952,0.015640,121.723618,0.096952,0.015640,121.723618,1,True,21
9,ExtraTrees_r42_BAG_L1,-1.220821e+00,-3.316153,root_mean_squared_error,0.106155,0.195372,0.963074,0.106155,0.195372,0.963074,1,True,20


In [ ]:
prediction = predictor.predict(test_df.drop(columns=['target']))

/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")
/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you t

In [ ]:
r2_score(test_df['target'],prediction)

0.878150048029356